In [2]:
# Allow auto-reload of external modules 
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import scipy as scp
import scipy.misc

import numpy as np
import tensorflow as tf


from network.fcn_vgg16 import FCN16VGG
import data_utils as dt


# Test 1

In [4]:
train_data_config = {'voc_dir':"data/VOC2012",
          'dataset':'train',
          'randomize': True,
          'seed': None}
params = {'num_classes': 22, 'rate': 1e-4}

train_dataset = dt.VOCDataSet(train_data_config)
next_pair = train_dataset.next_batch()
num_classes = params['num_classes'] 
label = next_pair[1]
num_pixels = label.shape[2]*label.shape[3]                  
label[label == 255] = num_classes - 1
label = label.reshape(label.shape[2],label.shape[3])


label_ = np.ones(label.shape)
label_[label == (num_classes - 1)] = 0
vec_label = np.reshape(label_,(num_pixels,))
print(vec_label.shape)
#print(np.diag(vec_label).shape)
diag_mul = tf.constant(vec_label,dtype=tf.int32)

# bias_add
bias_ = np.zeros(label.shape)
bias_[label == (num_classes - 1)] = 1  
vec_label_ = np.reshape(bias_,(num_pixels,))
bias_vec = tf.constant(vec_label_,dtype=tf.int32)


Indices loaded: 1464
Batch index: 448
(166500,)


In [ ]:
# Simulate train()
bias_add = tf.one_hot(bias_vec, depth=22, on_value=1,off_value=0,axis=-1)
bias_mul = tf.diag(diag_mul) 
bias_add = tf.cast(bias_add, tf.float32)
bias_mul = tf.cast(bias_mul, tf.float32)

prediction = tf.add(tf.matmul(bias_mul, bias_add),bias_add)
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    print(sess.run(bias_vec)[5:7])
    print(sess.run(bias_add)[5:7,:])
    print(sess.run(bias_add).shape)
    m = sess.run(prediction)    

[0 0]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.]]
(164268, 22)


In [142]:
print m

Tensor("one_hot_12:0", shape=(166500, 22), dtype=int32)


# Test 2

In [3]:
x = tf.random_uniform([3,3], minval=0, maxval=1, dtype=tf.float32)
# d = 4, c =3
y = np.array([[0,1,0],
              [0,1,0],
              [0,0,1],
              [1,0,0]])
indices = tf.constant([1,1,2,0])
y_ = tf.Variable(y)
yt = tf.one_hot(indices, depth=3, on_value=1.0, off_value=0.0, axis=-1)

#yt = tf.transpose(y_)
#last = tf.slice(yt, )
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    print(sess.run(yt))
    #xval,yval = sess.run([x,y_])


[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]]


In [101]:
# bias_add
bias_ = np.zeros(label.shape)
bias_[label == (num_classes - 1)] = 1  
vec_label_ = np.reshape(bias_,(num_pixels,1))
bias_add = tf.constant(vec_label_)
print(bias_add)

Tensor("Const_34:0", shape=(161000, 1), dtype=float64)


In [45]:
l = np.array([1,1,2])
y = np.array([[0,1,0],[0,1,0],[0,0,1]])
yc = np.copy(y)
y[l!=2,:] = 1 
y[l==2,:] = 0
yc[l!=2,:]= 0
z = xval * y + yc

In [42]:
l = np.array([1,1,2])
y =  np.ones([3,3])
yc = np.zeros([3,3])

y[l==2,:] = 0
yc[l!=2,:]= l
z = xval * y + yc

In [81]:
x = tf.Variable([[ 0.24924435,0.57862125, 0.24622613],
 [ 0.45910273,0.86502683,0.38240533],
 [ 0.82418419,0.29564904,0.92826386]], dtype=tf.float64)

In [84]:
def ignore_boundary(label,truth, num_classes):
    # Hot-one label 
    y = np.zeros((label.shape[0],num_classes))
    y[np.arange(num_classes), label] = 1

    yc = np.copy(y)
    c_max = num_classes - 1
    y[label != c_max,:] = 1 
    y[label == c_max,:] = 0
    yc[label != c_max,:]= 0
    yt = tf.constant(y)
    yct = tf.constant(yc)
    truth = tf.add(tf.mul(truth, yt), yct)
    return truth

l = np.array([1,1,2],dtype=np.int32)
# x = tf.random_uniform([3,3], minval=0, maxval=1, dtype=tf.float64)

truth = ignore_boundary(l, x,3)
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    print(sess.run(x))
    print(sess.run(truth))

[[ 0.24924435  0.57862125  0.24622613]
 [ 0.45910273  0.86502683  0.38240533]
 [ 0.82418419  0.29564904  0.92826386]]
[[ 0.24924435  0.57862125  0.24622613]
 [ 0.45910273  0.86502683  0.38240533]
 [ 0.          0.          1.        ]]
